### Using Word to Vector

In [1]:
import numpy as np # linear algebra
import pandas as pd

### Reading the data and removing missing values

In [2]:
from sklearn.model_selection import train_test_split

def read_data():                                        # creating a function named read_data()
    df = pd.read_csv("train.csv", nrows=20000)
    print ("Shape of base training File = ", df.shape)
    df.drop_duplicates(inplace=True)                    # Remove missing values and duplicates from training data
    df.dropna(inplace=True)
    print("Shape of base training data after cleaning = ", df.shape)
    return df
df = read_data()
df_train, df_test = train_test_split(df, test_size = 0.02)
print (df_train.head(2))


Shape of base training File =  (20000, 6)
Shape of base training data after cleaning =  (20000, 6)
          id   qid1   qid2                                          question1  \
13695  13695  26277  26278  What is the expected cut off NTSE stage 1 up 2...   
8905    8905  17334   3572                   How can one learn Japanese well?   

                                           question2  is_duplicate  
13695  What is the expected NTSE stage 1 UP cut off?             1  
8905                     How did you learn Japanese?             0  


### Checking the duplicate values counts

In [3]:
from collections import Counter
import matplotlib.pyplot as plt
import operator

def eda(df):                                        # defining a function to see unique questionsssss
    print ("Duplicate Count = %d , Non Duplicate Count = %d" 
           %(df.is_duplicate.value_counts()[1],df.is_duplicate.value_counts()[0]))
    
    question_ids_combined = df.qid1.tolist() + df.qid2.tolist()
    
    print ("Unique Questions = %s" %(len(np.unique(question_ids_combined))))
    
    question_ids_counter = Counter(question_ids_combined) #is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values
    sorted_question_ids_counter = sorted(question_ids_counter.items(), key=operator.itemgetter(1))
    question_appearing_more_than_once = [i for i in question_ids_counter.values() if i > 1]
    print ("Count of Quesitons appearing more than once = %s" %(len(question_appearing_more_than_once)))
    
    
eda(df_train)
import nltk
nltk.download('stopwords')


Duplicate Count = 7334 , Non Duplicate Count = 12266
Unique Questions = 37063
Count of Quesitons appearing more than once = 1733
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Creating functions to tokenize words in the question and filtering extremes

In [4]:
import re
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.porter import *

words = re.compile(r"\w+",re.I)         # creating an object and ignoring the case for matching and searching of words.
stopword = stopwords.words('english')
stemmer = PorterStemmer()               #  is a process for removing the commoner morphological and inflexional endings from words in English.

def tokenize_questions(df):
    question_1_tokenized = []
    question_2_tokenized = []

    for q in df.question1.tolist():
        question_1_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    for q in df.question2.tolist():
        question_2_tokenized.append([stemmer.stem(i.lower()) for i in words.findall(q) if i not in stopword])

    df["Question_1_tok"] = question_1_tokenized
    df["Question_2_tok"] = question_2_tokenized
    
    return df

def train_dictionary(df):
    
    questions_tokenized = df.Question_1_tok.tolist() + df.Question_2_tok.tolist()
    
    dictionary = corpora.Dictionary(questions_tokenized)  #able to store a collection of raw text documents with additional key-value headers.
    dictionary.filter_extremes(no_below=5, no_above=0.8)
    dictionary.compactify()                               # Assigns new ids to the word
    
    return dictionary
# Calling Function
df_train = tokenize_questions(df_train)
dictionary = train_dictionary(df_train)
print ("No of words in the dictionary = %s" %len(dictionary.token2id))

df_test = tokenize_questions(df_test)

C:\Users\rohit\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\rohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\rohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


No of words in the dictionary = 4838


### Creating a function named get_vector to convert documents into bag of words.

In [5]:
def get_vectors(df, dictionary):
    
    question1_vec = [dictionary.doc2bow(text) for text in df.Question_1_tok.tolist()] #Convert document into the bag-of-words
    question2_vec = [dictionary.doc2bow(text) for text in df.Question_2_tok.tolist()]
    
    question1_csc = gensim.matutils.corpus2csc(question1_vec, num_terms=len(dictionary.token2id))
    question2_csc = gensim.matutils.corpus2csc(question2_vec, num_terms=len(dictionary.token2id))
    
    return question1_csc.transpose(),question2_csc.transpose()


q1_csc, q2_csc = get_vectors(df_train, dictionary)

print (q1_csc.shape)
print (q2_csc.shape)

(19600, 4838)
(19600, 4838)


### Calculating different similarity values

In [6]:
from sklearn.metrics.pairwise import cosine_similarity as cs
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics import jaccard_similarity_score as jsc
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler

# https://en.wikipedia.org/wiki/Minkowski_distance 
# https://en.wikipedia.org/wiki/Cosine_similarity

minkowski_dis = DistanceMetric.get_metric('minkowski')
mms_scale_man = MinMaxScaler()
mms_scale_euc = MinMaxScaler()
mms_scale_mink = MinMaxScaler()

def get_similarity_values(q1_csc, q2_csc): # defining a function to calculate similarity values
    cosine_sim = []                        
    manhattan_dis = []
    eucledian_dis = []
    jaccard_dis = []
    minkowsk_dis = []
    
    for i,j in zip(q1_csc, q2_csc):
        sim = cs(i,j)
        cosine_sim.append(sim[0][0])
        sim = md(i,j)
        manhattan_dis.append(sim[0][0])
        sim = ed(i,j)
        eucledian_dis.append(sim[0][0])
        i_ = i.toarray()
        j_ = j.toarray()
        try:
            sim = jsc(i_,j_)
            jaccard_dis.append(sim)
        except:
            jaccard_dis.append(0)
            
        sim = minkowski_dis.pairwise(i_,j_)
        minkowsk_dis.append(sim[0][0])
    
    return cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis    


# cosine_sim = get_cosine_similarity(q1_csc, q2_csc)
cosine_sim, manhattan_dis, eucledian_dis, jaccard_dis, minkowsk_dis = get_similarity_values(q1_csc, q2_csc)
print ("cosine_sim sample= \n", cosine_sim[0:2])
print ("manhattan_dis sample = \n", manhattan_dis[0:2])
print ("eucledian_dis sample = \n", eucledian_dis[0:2])
print ("jaccard_dis sample = \n", jaccard_dis[0:2])
print ("minkowsk_dis sample = \n", minkowsk_dis[0:2])

eucledian_dis_array = np.array(eucledian_dis).reshape(-1,1)
manhattan_dis_array = np.array(manhattan_dis).reshape(-1,1)
minkowsk_dis_array = np.array(minkowsk_dis).reshape(-1,1)
    
manhattan_dis_array = mms_scale_man.fit_transform(manhattan_dis_array)
eucledian_dis_array = mms_scale_euc.fit_transform(eucledian_dis_array)
minkowsk_dis_array = mms_scale_mink.fit_transform(minkowsk_dis_array)

eucledian_dis = eucledian_dis_array.flatten()
manhattan_dis = manhattan_dis_array.flatten()
minkowsk_dis = minkowsk_dis_array.flatten()


cosine_sim sample= 
 [0.85714285714285687, 0.7745966692414834]
manhattan_dis sample = 
 [2.0, 2.0]
eucledian_dis sample = 
 [1.4142135623730951, 1.4142135623730951]
jaccard_dis sample = 
 [0.75, 0.59999999999999998]
minkowsk_dis sample = 
 [1.4142135623730951, 1.4142135623730951]


### Calculating the log loss values for the similarity values calculated above

In [7]:
from sklearn.metrics import log_loss

def calculate_logloss(y_true, y_pred):     # defining a function to calculate logloss
    loss_cal = log_loss(y_true, y_pred)
    return loss_cal

q1_csc_test, q2_csc_test = get_vectors(df_test, dictionary)
y_pred_cos, y_pred_man, y_pred_euc, y_pred_jac, y_pred_mink = get_similarity_values(q1_csc_test, q2_csc_test)
y_true = df_test.is_duplicate.tolist()

y_pred_man_array = mms_scale_man.transform(np.array(y_pred_man).reshape(-1,1))
y_pred_man = y_pred_man_array.flatten()

y_pred_euc_array = mms_scale_euc.transform(np.array(y_pred_euc).reshape(-1,1))
y_pred_euc = y_pred_euc_array.flatten()

y_pred_mink_array = mms_scale_mink.transform(np.array(y_pred_mink).reshape(-1,1))
y_pred_mink = y_pred_mink_array.flatten()

logloss = calculate_logloss(y_true, y_pred_cos)
print ("The calculated log loss value on the test set for cosine sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_man)
print ("The calculated log loss value on the test set for manhattan sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_euc)
print ("The calculated log loss value on the test set for euclidean sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_jac)
print ("The calculated log loss value on the test set for jaccard sim is = %f" %logloss)

logloss = calculate_logloss(y_true, y_pred_mink)
print ("The calculated log loss value on the test set for minkowski sim is = %f" %logloss)

The calculated log loss value on the test set for cosine sim is = 1.569353
The calculated log loss value on the test set for manhattan sim is = 2.327256
The calculated log loss value on the test set for euclidean sim is = 1.964779
The calculated log loss value on the test set for jaccard sim is = 3.265609
The calculated log loss value on the test set for minkowski sim is = 1.964779


### Looking at the parameters of Support Vector Regressor

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_train = pd.DataFrame({"cos" : cosine_sim, "man" : manhattan_dis, "euc" : eucledian_dis, "jac" : jaccard_dis, "min" : minkowsk_dis})
y_train = df_train.is_duplicate

X_test = pd.DataFrame({"cos" : y_pred_cos, "man" : y_pred_man, "euc" : y_pred_euc, "jac" : y_pred_jac, "min" : y_pred_mink})
y_test = y_true

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

svr = SVR()
svr.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Calculating log loss values using Random Forest Regressor and Support Vector Regressor

In [9]:
# https://en.wikipedia.org/wiki/Random_forest
# http://www.saedsayad.com/support_vector_machine_reg.htm
y_rfr_predicted = rfr.predict(X_test)
y_svr_predicted = svr.predict(X_test)

logloss_rfr = calculate_logloss(y_test, y_rfr_predicted)
logloss_svr = calculate_logloss(y_test, y_svr_predicted)

print ("The calculated log loss value on the test set using RFR is = %f" %logloss_rfr)
print ("The calculated log loss value on the test set using SVR is = %f" %logloss_svr)

The calculated log loss value on the test set using RFR is = 1.232292
The calculated log loss value on the test set using SVR is = 0.696790


The text in the document by <Shireen Rabbani, Rohit Ulhe> is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/
The code in the document by <Shireen Rabbani, Rohit Ulhe> is licensed under the MIT License https://opensource.org/licenses/MIT